# About: Moodle環境の削除--VCノードの削除

---

VCノードを削除します。

## パラメータの設定

構築環境を削除するのに必要となるパラメータを設定します。

### Ansibleのグループを指定する

削除対象となるAnsibleのグループ名を指定してください。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定された `target_group` の値が適切なものかチェックします。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

### VCCアクセストークンの入力

VCC のアクセストークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2020-04-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2020-04-XX XX:XX:XX,XXX - ERROR - 2020/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

この場合はアクセストークンの入力からやり直してください。

## VCノードの削除

Moodle環境のVCノードを削除します。

まず削除対象となるUnitGroupの現在の状態を確認します。

In [ ]:
ugroup = vcp.get_ugroup(target_group)
ugroup.df_nodes()

UnitGroupの一覧も確認しておきます。

In [ ]:
vcp.df_ugroups()

VCノードを削除します。構築したMoodle環境が**削除**されます。

In [ ]:
ugroup.cleanup()

削除後の UnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

## Ansible 設定ファイルの後始末

Ansibleに関する設定ファイルの後始末を行います。

group_vars に記録されているパラメータを無効にするために group_vars ファイルをリネームします。

In [ ]:
!mv group_vars/{target_group}  group_vars/.{target_group}.$(date +"%Y%m%d%H%M%S").bak

インベントリから対象となるグループを削除します。

In [ ]:
%run scripts/group.py
remove_group_from_inventory_yml(target_group)

# 削除前との差分を表示します。
!diff -u inventory.yml.bak inventory.yml || true